In [1]:
# Change to working directory to enable caching during training for faster training
!cp -r /kaggle/input/bdd10k-road-segmentation-yolo/BDDk /kaggle/working/

In [2]:
!pip install -q ultralytics albumentations

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.5/974.5 kB 3.6 MB/s eta 0:00:00


In [3]:
dataset_yaml = """
train: /kaggle/working/BDDk/images/train
val: /kaggle/working/BDDk/images/val

nc: 1
names:
  0: road

"""
with open("/kaggle/working/dataset.yaml", "w") as f:
    f.write(dataset_yaml)

print("dataset.yaml updated for segmentation!")


dataset.yaml updated for segmentation!


In [4]:
import os
import random
import shutil
import cv2
import albumentations as A
from glob import glob

# ✅ Paths (adjust as necessary)
IMG_DIR = '/kaggle/working/BDDk/images/train'
LBL_DIR = '/kaggle/working/BDDk/labels/train'

# ✅ Albumentations pipeline
augment = A.Compose([
    A.OneOf([
        A.GaussianBlur(blur_limit=(3, 7), p=0.5),
        A.MotionBlur(blur_limit=5, p=0.5),
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
    ], p=0.6),
    A.RandomBrightnessContrast(p=0.4),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=30, val_shift_limit=20, p=0.4)
])

# ✅ Load image paths
all_images = glob(os.path.join(IMG_DIR, '*.jpg'))
random.shuffle(all_images)
subset = all_images[:int(len(all_images) * 0.10)]  # 10% augmentation

# ✅ Apply augmentation to subset and save back to original folder
for img_path in subset:
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    aug_img = augment(image=img_rgb)['image']
    aug_img_bgr = cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR)

    # Save augmented image with '_aug' suffix in the same directory
    base = os.path.basename(img_path).replace('.jpg', '')
    aug_img_path = os.path.join(IMG_DIR, f'{base}_aug.jpg')
    cv2.imwrite(aug_img_path, aug_img_bgr)

    # Copy the corresponding label file with the '_aug' suffix
    lbl_path = os.path.join(LBL_DIR, f'{base}.txt')
    aug_lbl_path = os.path.join(LBL_DIR, f'{base}_aug.txt')
    shutil.copy(lbl_path, aug_lbl_path)

print("✅ Augmented 10% of dataset and saved them with '_aug' suffix in the same folders.")


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


✅ Augmented 10% of dataset and saved them with '_aug' suffix in the same folders.


In [5]:
import os
import torch
import gc
from ultralytics import YOLO

# ✅ GPU memory config
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

# ✅ Paths
model_path = "yolo11s-seg.pt"  # Your pretrained or base model
data_yaml = "/kaggle/working/dataset.yaml"
save_dir = "runs/segment/train"
last_ckpt = os.path.join(save_dir, "weights", "last.pt")

# ✅ Resume or fresh
resume_training = os.path.exists(last_ckpt)
print(f"{'🔁 Resuming' if resume_training else '🚀 Starting new'} training...")

# ✅ Load model
model = YOLO(last_ckpt if resume_training else model_path)
model.train(
    data=data_yaml,
    epochs=100,
    batch=4,
    imgsz=640,
    device=0,
    resume=resume_training,
    save_period=5,
    patience=10,
    cache=True,

    # ✅ YOLO augmentations
    hsv_h=0.015,
    hsv_s=0.6,
    hsv_v=0.4,
    degrees=5.0,
    translate=0.1,
    scale=0.4,
    shear=2.0,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,
    copy_paste=0.0
)
torch.cuda.empty_cache()
gc.collect()
print("🧹 Memory cleared after training.")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🚀 Starting new training...


100%|██████████| 19.7M/19.7M [00:00<00:00, 232MB/s]


Ultralytics 8.3.107 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=segment, mode=train, model=yolo11s-seg.pt, data=/kaggle/working/dataset.yaml, epochs=100, time=None, patience=10, batch=4, imgsz=640, save=True, save_period=5, cache=True, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True

100%|██████████| 755k/755k [00:00<00:00, 29.6MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  4                  -1  1    103360  ultralytics.nn.modules.block.C3k2            [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  7                  -1  1   1180672  ultralytics

...

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 125/125 [00:10<00:00, 11.41it/s]


                   all        999       3098      0.797      0.413       0.47      0.373       0.81       0.41      0.471      0.364


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.1ms preprocess, 5.3ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/segment/train
🧹 Memory cleared after training.


In [6]:
import shutil

# Zip and move output
shutil.make_archive("/kaggle/working/runs", 'zip', "runs")
print("✅ runs.zip ready to download!")

✅ runs.zip ready to download!
